In [1]:
#import yt_dlp
import os
import numpy as np
import librosa
import pandas as pd

In [2]:
import gc
import numpy as np
import librosa

# Monkey-patch numpy for librosa compatibility
if not hasattr(np, 'complex'):
    np.complex = complex

def extract_audio_features(file_path, song_id_txt, sr=None, hop_length=512):
    """
    Extrae características de un archivo de audio, sincronizadas con el beat,
    intentando incluir chroma_vqt con varios ajustes para no exceder Nyquist.
    """
    try:
        # Carga y separación
        y, sr = librosa.load(file_path, sr=sr, duration=120.0)
        y_harmonic, y_percussive = librosa.effects.hpss(y)
        tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)

        # Función para generar intervals y calcular chroma_vqt con fallback
        def compute_chroma_vqt(y_h, sr, hop_length):
            for bins in [12, 8, 6, 4]:
                try:
                    fmin = librosa.note_to_hz('C1')
                    intervals = librosa.cqt_frequencies(
                        n_bins=bins,
                        fmin=fmin,
                        bins_per_octave=bins
                    )
                    return librosa.feature.chroma_vqt(
                        y=y_h, sr=sr,
                        hop_length=hop_length,
                        intervals=intervals
                    )
                except Exception:
                    continue
            # Si todos fallan, lanzar
            raise RuntimeError("No se pudo calcular chroma_vqt con bins reducidos")

        # Calcular features raw
        raw_feats = {
            'chroma_stft':      librosa.feature.chroma_stft(y=y, sr=sr, hop_length=hop_length),
            'chroma_cqt':       librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=hop_length),
            'melspectrogram':   librosa.feature.melspectrogram(y=y, sr=sr, hop_length=hop_length),
            'mfcc':             librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13),
            'mfcc_delta':       librosa.feature.delta(
                                     librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)),
            'rms':              librosa.feature.rms(y=y, hop_length=hop_length),
            'spectral_centroid':    librosa.feature.spectral_centroid(y=y, sr=sr, hop_length=hop_length),
            'spectral_bandwidth':   librosa.feature.spectral_bandwidth(y=y, sr=sr, hop_length=hop_length),
            'spectral_contrast':    librosa.feature.spectral_contrast(y=y, sr=sr, hop_length=hop_length),
            'spectral_flatness':    librosa.feature.spectral_flatness(y=y, hop_length=hop_length),
            'spectral_rolloff':     librosa.feature.spectral_rolloff(y=y, sr=sr, hop_length=hop_length),
            'tonnetz':              librosa.feature.tonnetz(y=y_harmonic, sr=sr),
            'zero_crossing_rate':   librosa.feature.zero_crossing_rate(y, hop_length=hop_length)
        }

        # Intentar añadir chroma_vqt
        try:
            raw_feats['chroma_vqt'] = compute_chroma_vqt(y_harmonic, sr, hop_length)
        except Exception as e:
            print(f"  ⚠️ Omitiendo chroma_vqt: {e}")

        # Sincronizar cada feature con beat
        result = {'song_id': song_id_txt.replace(".txt", ""), 'tempo': float(tempo)}
        for name, F in raw_feats.items():
            synced = librosa.util.sync(F, beat_frames, aggregate=np.mean)
            result[f"{name}_beat"] = synced.T.tolist()

        return result

    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

    finally:
        gc.collect()

# Verificación
print("Función lista con fallback para chroma_vqt.")


Función lista con fallback para chroma_vqt.


In [3]:
import os
import yt_dlp

folder = "Canciones2"
os.makedirs(folder, exist_ok=True)

PO_TOKEN = "MlO6fT_1PrFAPrpBEN2LA0d4vPt_We2gjf4TdmOFjd-49oyE3aF45hsf6iafDUygEBrWdwYee1ZFKcNzaE17fUQEdyR6CUCKV29UHCI6mBjeBFykPA=="

def download_audio(search_query, output_filename):
    ydl_opts = {
        'format': 'bestaudio/best',
        'cookiefile': 'cookiesY.txt',
        'outtmpl': os.path.join(folder, output_filename + '.wav'),
        'noplaylist': True,
        'default_search': 'ytsearch',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
        }],
        'extractor_args': {
            'youtube': f'player_client=web;po_token=web.gvs+{PO_TOKEN}'
        },
        'ignore_no_formats_error': True,
    }

    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([search_query])
        print(f"✅ Descargado: {output_filename}.wav")
    except yt_dlp.utils.DownloadError as de:
        print(f"❌ DownloadError al descargar '{search_query}': {de}")
    except Exception as e:
        print(f"❌ Error inesperado al descargar '{search_query}': {e}")

# Ejemplo de uso:
# download_audio("Cold As Ice Atlas Sound", "cold-as-ice")


In [4]:
df = pd.read_csv("Data_completa_enrriquecida.csv")

In [5]:
df.head(5)

,Letra,Idioma,lyrics_state,hasLetra,Lyrics,Lyrics_proces,songId,track_uri,track_name,artist_uri,...,dominance_tags,mbid,spotify_id,genre,cuadrante,cuadReal,lexicones,emociones,emocionesLetra,emocion_mas_comun
0,Data/Letras/1-'Till-I-Collapse-Eminem.txt,en,complete,1.0,TranslationsEspañolTürkçePortuguêsItalianoहिन्...,cause feel tired left left feel weak feel weak...,till-i-collapse-eminem.txt,spotify:track:4xkOaSrkexMciUUogZKVTS,Till I Collapse,spotify:artist:7dGJo4pcD2V6oG8kP0tJRR,...,5.690625,cab93def-26c5-4fb0-bedd-26ec4c1619e1,4xkOaSrkexMciUUogZKVTS,rap,3,2,[0. 2. 0. 0. 0. 2. 6. 0. 0. 2.],"['anticipation', 'negative', 'positive', 'trust']","['3', '2', '1', '3']",3
1,Data/Letras/2-St.-Anger-Metallica.txt,en,complete,1.0,St. Anger Lyrics[Verse]\nSaint Anger 'round my...,saint anger round neck saint anger round neck ...,st.-anger-metallica.txt,spotify:track:3fOc9x06lKJBhz435mInlH,St. Anger,spotify:artist:2ye2Wgw4gimLv2eAKyk1NB,...,5.427250,727a2529-7ee8-4860-aef6-7959884895cb,3fOc9x06lKJBhz435mInlH,metal,3,2,[0. 6. 0. 0. 0. 6. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
2,Data/Letras/3-Speedin'-Rick-Ross.txt,en,complete,1.0,Speedin’ Lyrics[Intro: Rick Ross]\nLegendary\n...,legendary runners know trilla dollar count acc...,speedin-rick-ross.txt,spotify:track:3Y96xd4Ce0J47dcalLrEC8,Speedin',spotify:artist:1sBkRIssrMs1AbVkOJbc7a,...,5.490000,NaN,3Y96xd4Ce0J47dcalLrEC8,rap,3,2,[0. 2. 0. 0. 0. 2. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3
3,Data/Letras/4-Bamboo-Banga-M.I.A..txt,en,complete,1.0,"Bamboo Banga Lyrics[Intro]\nRoad runner, road ...",road runner road runner going miles hour road ...,bamboo-banga-m.i.a..txt,spotify:track:6tqFC1DIOphJkCwrjVzPmg,Bamboo Banga,spotify:artist:0QJIPDAEDILuo8AIq3pMuU,...,5.691357,99dd2c8c-e7c1-413e-8ea4-4497a00ffa18,6tqFC1DIOphJkCwrjVzPmg,hip-hop,3,1,[0. 0. 0. 0. 0. 0. 4. 0. 0. 0.],['positive'],['1'],1
4,Data/Letras/5-Die-MF-Die-Dope.txt,en,complete,1.0,Die MF Die Lyrics[Intro]\nDie!\n\n[Verse 1]\nI...,die need forgiveness need hate need acceptance...,die-mf-die-dope.txt,spotify:track:5bU4KX47KqtDKKaLM4QCzh,Die MF Die,spotify:artist:7fWgqc4HJi3pcHhK8hKg2p,...,5.441765,b9eb3484-5e0e-4690-ab5a-ca91937032a5,5bU4KX47KqtDKKaLM4QCzh,metal,3,1,[0. 1. 0. 0. 0. 1. 0. 0. 0. 0.],"['anticipation', 'negative']","['3', '2']",3


In [6]:
df.shape

(24966, 50)

In [7]:
def single_or_mode(series):
    """
    Si la columna tiene exactamente un único valor en el grupo, lo devuelve.
    De lo contrario, devuelve la moda (el valor más frecuente).
    En caso de no existir valor definido, devuelve None.
    """
    sin_nan = series.dropna()
    
    # Valores únicos
    unique_vals = sin_nan.unique()
    
    if len(unique_vals) == 1:
        # Si hay un único valor, lo devolvemos
        return unique_vals[0]
    else:
        modos = sin_nan.mode()
        if len(modos) > 0:
            return modos.iloc[0]
        else:
           
            return None


In [8]:
# Función para manejar duplicados de songId
def consolidate_group(group):
    return pd.Series({
        'Idioma': single_or_mode(group['Idioma']),
        'lyrics_state': single_or_mode(group['lyrics_state']),
        'track_name':single_or_mode(group['track_name']),
        'artist_uri':single_or_mode(group['artist_uri']),
        'hasLetra': group['hasLetra'].max(),
        'Lyrics': max(group['Lyrics'], key=len),
        'Lyrics_proces': max(group['Lyrics_proces'], key=len),
        'track_uri': single_or_mode(group['track_uri']),
        'valence': single_or_mode(group['valence']),
        'valence_tags': single_or_mode(group['valence_tags']),
        'arousal_tags': single_or_mode(group['arousal_tags']),
        'emocion_mas_comun': single_or_mode(group['emocion_mas_comun']),
        'cuadrante': single_or_mode(group['cuadrante']),
        'cuadReal': single_or_mode(group['cuadReal'])
    })



df = df.groupby('songId').apply(consolidate_group).reset_index()
print("Número de songIds únicos después de consolidar:", df['songId'].nunique())

Número de songIds únicos después de consolidar: 24026


C:\Users\andre\AppData\Local\Temp\ipykernel_15620\1057465179.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('songId').apply(consolidate_group).reset_index()


In [9]:
song_id_counts = df['songId'].value_counts()

duplicated_song_ids = song_id_counts[song_id_counts > 1].index

df[df['songId'].isin(duplicated_song_ids)].head(20)

,songId,Idioma,lyrics_state,track_name,artist_uri,hasLetra,Lyrics,Lyrics_proces,track_uri,valence,valence_tags,arousal_tags,emocion_mas_comun,cuadrante,cuadReal


In [12]:
import os
import pandas as pd

# --- Parámetros de entrada ---
audio_folder = "Canciones2"
feat_csv     = "features_Librosa_complete.csv"

# --- 1. Listar los .wav existentes en Canciones2 ---
existing_audio = {
    os.path.splitext(f)[0]
    for f in os.listdir(audio_folder)
    if f.lower().endswith(".wav")
}

# --- 2. Leer el CSV de features ya generado ---
feats_df = pd.read_csv(feat_csv)
existing_feats = set(feats_df['song_id'].astype(str))

# --- 3. Preparar tu DataFrame de canciones ---
df['audio_file'] = df['songId'].str.replace(".txt", "", regex=False) + ".wav"
df['audio_base'] = df['audio_file'].str[:-4]

# --- 4. Marcar presencia en carpeta y en CSV de features ---
df['in_folder']  = df['audio_base'].isin(existing_audio)
df['in_librosa'] = df['audio_base'].isin(existing_feats)

# --- 5. Eliminar la clase cuadrante 0 ---
df = df[df['cuadrante'] != 0]

# --- 6. Filtrar sólo lo que está en carpeta pero no en Librosa ---
to_process = df[df['in_folder'] & ~df['in_librosa']].copy()

# --- 7. Estadísticas por cuadrante (sin el 0) ---
print("\nCanciones en carpeta por cuadrante:")
print(df[df['in_folder']].groupby('cuadrante').size().to_frame("en_folder"))

print("\nCanciones fuera de carpeta por cuadrante:")
print(df[~df['in_folder']].groupby('cuadrante').size().to_frame("faltantes"))

print("\nCanciones EN carpeta pero SIN features (a procesar):")
print(to_process.groupby('cuadrante').size().to_frame("por_procesar"))

# `to_process['audio_file']` es ahora la lista limpia de .wav a procesar,
# excluyendo todas las de cuadrante 0.



Canciones en carpeta por cuadrante:
           en_folder
cuadrante           
1               1061
2               1054
3               1048
4                942

Canciones fuera de carpeta por cuadrante:
           faltantes
cuadrante           
1               3496
2               2263
3               1543
4               1443

Canciones EN carpeta pero SIN features (a procesar):
           por_procesar
cuadrante              
1                  1061
2                   919
3                   655
4                   156


In [10]:
import os
import pandas as pd

# 1. Listar los .wav que ya están en la carpeta "Canciones2"
folder = "Canciones2"
existing_files = set(f for f in os.listdir(folder) if f.lower().endswith(".wav"))
print(f"Total archivos en '{folder}':", len(existing_files))

# 2. Crear columna 'audio_file' en df
df['audio_file'] = df['songId'].str.replace(".txt", "", regex=False) + ".wav"

# 3. Marcar qué canciones ya tengo y cuáles faltan
df['in_folder'] = df['audio_file'].isin(existing_files)

# 4. Contar cuántas hay por cuadrante dentro vs fuera
counts_in  = df[df['in_folder']].groupby('cuadrante').size().sort_index()
counts_out = df[~df['in_folder']].groupby('cuadrante').size().sort_index()

print("\nCanciones ya en carpeta por cuadrante:")
print(counts_in.to_frame("en_folder"))
print("\nCanciones faltantes por cuadrante:")
print(counts_out.to_frame("faltantes"))


Total archivos en 'Canciones2': 4788

Canciones ya en carpeta por cuadrante:
           en_folder
cuadrante           
0                683
1               1061
2               1054
3               1048
4                942

Canciones faltantes por cuadrante:
           faltantes
cuadrante           
0              10493
1               3496
2               2263
3               1543
4               1443


In [11]:
# 1. Define cuántas quieres por cada cuadrante
n_per_quad = {
    2: 140,
    3: 400,
    4: 800
}

# 2. Construye la lista de DataFrames muestreados
dfs = []
for q, N in n_per_quad.items():
    # Filtrar sólo las que faltan y corresponden al cuadrante q
    df_q = df[(df['cuadrante'] == q) & (~df['in_folder'])]
    available = len(df_q)

    if available < N:
        print(f"¡Atención! Solo hay {available} canciones faltantes en el cuadrante {q}. Se tomarán todas.")
        dfs.append(df_q)  # tomo todas las disponibles
    else:
        dfs.append(df_q.sample(n=N, random_state=42))

# 3. Concatenar el resultado
df_to_download = pd.concat(dfs, ignore_index=True)
print(f"\nTotal canciones seleccionadas para descarga: {len(df_to_download)}")

# Opcional: mostrar cuántas se han tomado de cada cuadrante
print(df_to_download['cuadrante'].value_counts().sort_index())



Total canciones seleccionadas para descarga: 1340
cuadrante
2    140
3    400
4    800
Name: count, dtype: int64


In [12]:
#counts = df_not_in_folder['cuadReal'].value_counts()
#print(counts)

In [12]:
df_to_download["track_uri"].to_csv("uris.txt", index=False, header=False)

In [ ]:
import subprocess

OUTPUT_FOLDER = "Canciones2"

for i, row in df_to_download.iterrows():
    uri    = row["track_uri"]
    songid = row["songId"].replace(".txt", "")
    print(f"[{i+1}/{len(df_to_download)}] Descargando {songid!r}…")

    try:
        # No usamos check=True aquí, capturamos siempre la salida
        cp = subprocess.run(
            [
                "spotdl", "download", uri,
                "--format", "wav",
                "--output", f"{OUTPUT_FOLDER}/{songid}.{{output-ext}}"
            ],
            capture_output=True, text=True
        )

        if cp.returncode == 0:
            print(f"✅ {songid}.wav descargado correctamente")
        else:
            print(f"❌ Error descargando {songid}:")
            print(cp.stderr or cp.stdout)   # a veces sale por stdout
    except subprocess.CalledProcessError as e:
        # Si decides usar check=True de nuevo, atrápalo así:
        print(f"❌ CalledProcessError en {songid}:")
        print(e.stderr or e.output)
    except Exception as e:
        print(f"❌ Excepción inesperada en {songid}: {e}")

print("🏁 ¡Proceso completado!")

In [13]:
import os
import csv
import json

# 1) Definimos los nombres de las columnas que devuelve extract_audio_features
feature_names = [
    'chroma_stft_beat', 'chroma_cqt_beat', 'chroma_vqt_beat',
    'melspectrogram_beat', 'mfcc_beat', 'mfcc_delta_beat',
    'rms_beat', 'spectral_centroid_beat', 'spectral_bandwidth_beat',
    'spectral_contrast_beat', 'spectral_flatness_beat',
    'spectral_rolloff_beat', 'tonnetz_beat', 'zero_crossing_rate_beat'
]

# 2) Creamos el archivo con todas las columnas
with open("features_Librosa_complete2.csv", "w", newline="", encoding="utf-8") as f:
    # Lista de cabeceras: song_id, tempo + cada feature synchronizada
    fieldnames = ['song_id', 'tempo'] + feature_names
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()

    # 3) Iteramos sólo sobre las canciones que queremos procesar
    for _, row in to_process.iterrows():
        audio_path = os.path.join("Canciones2", row['audio_file'])
        if not os.path.isfile(audio_path):
            print(f"⚠️ Archivo no encontrado, se omite: {audio_path}")
            continue

        print(f"Extrayendo características de {audio_path}")
        features = extract_audio_features(audio_path, row['songId'])
        if features is None:
            continue

        # 4) Preparamos el diccionario de fila
        csv_row = {
            'song_id': features['song_id'],
            'tempo': features['tempo']
        }
        # Serializamos cada lista de beats como JSON
        for feat in feature_names:
            csv_row[feat] = json.dumps(features.get(feat, []))

        # 5) Escribimos la fila
        writer.writerow(csv_row)

        # Limpieza
        del features


Extrayendo características de Canciones2\(argument-with-david-rawlings-concerning-morrissey)-ryan-adams.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos


C:\Users\andre\AppData\Local\Temp\ipykernel_15620\1112523765.py:65: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  result = {'song_id': song_id_txt.replace(".txt", ""), 'tempo': float(tempo)}


Extrayendo características de Canciones2\(bloody-paw-on-the)-kill-floor-busdriver.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\(i-saw-santa)-rockin-around-christmas-tree-beach-boys.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\(what-a)-wonderful-world-art-garfunkel.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\006-mc-chris.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\1049-gotho-idles.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\12-baaba.wav
  ⚠️ Omitiendo chroma_vqt: No se pudo calcular chroma_vqt con bins reducidos
Extrayendo características de Canciones2\15-petals-elvis-costello.wav
  ⚠️ Omitiendo chroma_vqt